In [1]:
# Intalar pymupdf
# %pip install pymupdf

#importar fitz para usar pymupdf (fitz es pymupdf)
import fitz  

In [2]:
# Instalar e importar la biblioteca pendulum para trabajar con fechas
# %pip install pendulum

import pendulum

# Definir timezone e idioma en la biblioteca pendulum

tz = pendulum.timezone('Europe/Paris')
pendulum.set_locale('es')

In [22]:
# Importar regex
import re

In [24]:
# Intalar e importar la biblioteca para trabajar con codigos postales
# %pip install pgeocode

import pgeocode

In [3]:
def formato_fecha(fecha):
    dt = ''
    if '/' in fecha:
        dt = pendulum.from_format(fecha, 'DD/MM/YYYY')
    elif 'de' in fecha:
        dt = pendulum.from_format(fecha.replace(' de', ''), 'DD MMMM YYYY')
    try:
        dt_final = dt.format('DD.MM.YYYY')
    except Exception:
        dt_final = 'fecha'  
    return dt_final    


In [4]:
# Extraer el texto de cada página como string en una lista 
file = fitz.open("pdfs/factura_555.pdf")
pymupdf_text = []

for page in file:
    pymupdf_text.append(page.get_text())

print(pymupdf_text)    

['Página 1 / 2 \nGDF SUEZ TRADING \nCIF BE0835490593 \nDomicilio social: 1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400 \n- Extranjero \nRemite: GDF SUEZ TRADING Apartado de Correos XXXXX 92400 PARIS (Francia) \nNIEVES FRÍAS SARDINAS \n \nCalle los Jabalíes \n \n43595 Prat de Comte \nDocumento emitido por GDF SUEZ TRADING – 1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400 - Extranjero; inscrita en el Registro Mercantil de Extranjero, tomo 15716, folio 34, sección 2, hoja número M-95971, inscripción 47 - CIF \nBE0835490593 \nINFORMACIÓN DEL CONSUMO ELÉCTRICO \nEVOLUCIÓN DE CONSUMO \n100 \n \n \n75 \n \n \n50 \n \n \n25 \n \n \nkWh \nJn.  \nJl.   Ag.   St.   Oc.  Nv.  Dc.   En.   Fb.   Mr.  Ab.    My.     Jn.   Jl.   Ag. \n18  \n18  18   18  18   18   18   19   19   19   19   19   19   19  19 \n \n \n \n \n \nSu consumo medio diario en este último periodo facturado ha sido: 0,XX € \nSu consumo medio diario en los últimos 14 meses ha sido: 0,XX € \nSu consumo acumulado del último año ha

In [5]:
""" En caso de necesitar el texto de todas las páginas del pdf 
lineas_pdf = []
for x in range(len(pymupdf_text)):
    lineas = pymupdf_text[x].split('\n')
    lineas_pdf.append(lineas)
lineas_pdf 
""" 

" En caso de necesitar el texto de todas las páginas del pdf \nlineas_pdf = []\nfor x in range(len(pymupdf_text)):\n    lineas = pymupdf_text[x].split('\n')\n    lineas_pdf.append(lineas)\nlineas_pdf \n"

In [6]:
# Crear el diccionario que se convertirá en el archivo JSON final con los datos a conseguir
diccionario_final = {
    "nombre_cliente": "",
    "dni_cliente": "",
    "calle_cliente": "",
    "cp_cliente": "",
    "población_cliente": "",
    "provincia_cliente": "",
    "nombre_comercializadora": "",
    "cif_comercializadora": "",
    "dirección_comercializadora": "",
    "cp_comercializadora": "",
    "población_comercializadora": "",
    "provincia_comercializadora": "",
    "número_factura": "",
    "inicio_periodo": "",
    "fin_periodo": "",
    "importe_factura": "",
    "fecha_cargo": "",
    "consumo_periodo": "",
    "potencia_contratada": ""
}

In [7]:
# Extraer el texto de la primera página del pdf (dónde está toda la información necesaria)
lineas_pag1 = pymupdf_text[0].split('\n')

In [8]:
# Se limpian los datos para eliminar las líneas vacias.
# Se crea un diccionario con los apartados del pdf del que se pueden extraer pares key/valor mediante la separación por los ':'.
# Se crea una lista con todos los datos para tenerlos ordenados.

dict_lineas = {}
lista_lineas = []
contador = 0
for linea in lineas_pag1:
    linea = linea.strip()
    #Se eliminan lineas vacias (o de solo un espacio)
    if linea != '' and linea != ' ':
        lista_lineas.append(linea)
        #Se dividen las líneas por el primer ':' para separarlas en Key/valor en el diccionario
        if ':' in linea:
            lista_linea = linea.split(':', 1)
            dict_lineas[lista_linea[0]] = lista_linea[1].strip() 
        else:
            # En caso de no contar con ':' en la línea, se añade al diccionario con la key?otro' más un número incremental
            contador += 1
            dict_lineas[f'otro{contador}'] = linea

dict_lineas         

{'otro1': 'Página 1 / 2',
 'otro2': 'GDF SUEZ TRADING',
 'otro3': 'CIF BE0835490593',
 'Domicilio social': '1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400',
 'otro4': '- Extranjero',
 'Remite': 'GDF SUEZ TRADING Apartado de Correos XXXXX 92400 PARIS (Francia)',
 'otro5': 'NIEVES FRÍAS SARDINAS',
 'otro6': 'Calle los Jabalíes',
 'otro7': '43595 Prat de Comte',
 'otro8': 'Documento emitido por GDF SUEZ TRADING – 1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400 - Extranjero; inscrita en el Registro Mercantil de Extranjero, tomo 15716, folio 34, sección 2, hoja número M-95971, inscripción 47 - CIF',
 'otro9': 'BE0835490593',
 'otro10': 'INFORMACIÓN DEL CONSUMO ELÉCTRICO',
 'otro11': 'EVOLUCIÓN DE CONSUMO',
 'otro12': '100',
 'otro13': '75',
 'otro14': '50',
 'otro15': '25',
 'otro16': 'kWh',
 'otro17': 'Jn.',
 'otro18': 'Jl.   Ag.   St.   Oc.  Nv.  Dc.   En.   Fb.   Mr.  Ab.    My.     Jn.   Jl.   Ag.',
 'otro19': '18',
 'otro20': '18  18   18  18   18   18   19   19   19   19   19   19 

In [9]:
lista_lineas

['Página 1 / 2',
 'GDF SUEZ TRADING',
 'CIF BE0835490593',
 'Domicilio social: 1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400',
 '- Extranjero',
 'Remite: GDF SUEZ TRADING Apartado de Correos XXXXX 92400 PARIS (Francia)',
 'NIEVES FRÍAS SARDINAS',
 'Calle los Jabalíes',
 '43595 Prat de Comte',
 'Documento emitido por GDF SUEZ TRADING – 1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400 - Extranjero; inscrita en el Registro Mercantil de Extranjero, tomo 15716, folio 34, sección 2, hoja número M-95971, inscripción 47 - CIF',
 'BE0835490593',
 'INFORMACIÓN DEL CONSUMO ELÉCTRICO',
 'EVOLUCIÓN DE CONSUMO',
 '100',
 '75',
 '50',
 '25',
 'kWh',
 'Jn.',
 'Jl.   Ag.   St.   Oc.  Nv.  Dc.   En.   Fb.   Mr.  Ab.    My.     Jn.   Jl.   Ag.',
 '18',
 '18  18   18  18   18   18   19   19   19   19   19   19   19  19',
 'Su consumo medio diario en este último periodo facturado ha sido: 0,XX €',
 'Su consumo medio diario en los últimos 14 meses ha sido: 0,XX €',
 'Su consumo acumulado del último año h

In [15]:
# Se obtienen todos los datos que se pueden extraer mediante las llaves del diccionario 
for key in dict_lineas.keys():
    if 'importe' in key.lower() and 'factura' in key.lower() and 'total' not in key.lower():
        print(f'{key}: {dict_lineas[key]}')
        diccionario_final["importe_factura"] = dict_lineas[key].split()[0]
    if 'nº' in key.lower():
        diccionario_final["número_factura"] = dict_lineas[key]
    if 'periodo' in key.lower():
        fechas_consumo = dict_lineas[key].split(' a ')
        print(fechas_consumo)
        diccionario_final["inicio_periodo"] = formato_fecha(fechas_consumo[0].strip())
        diccionario_final["fin_periodo"] = formato_fecha(fechas_consumo[-1].strip())
    if 'fecha' in key.lower() and 'cargo' in key.lower():
        diccionario_final["fecha_cargo"] = formato_fecha(dict_lineas[key])
    if 'potencia' in key.lower():
        diccionario_final["potencia_contratada"] = dict_lineas[key].split()[0]     
    if 'titular' in key.lower():
        diccionario_final["nombre_cliente"] = dict_lineas[key]     
    if 'domicilio social' in key.lower():
        diccionario_final["provincia_comercializadora"] = dict_lineas[key].split('-')[-1].strip()   

# Se duplica el for en el caso del cif y el Nif para poder romper el loop después de la primera localización y asegurarnos que es el de la comercializadora y el cliente respectivamente

for key in dict_lineas.keys():
    if 'cif' in key.lower():
        diccionario_final["cif_comercializadora"] = dict_lineas[key]  
        break 

for key in dict_lineas.keys():
    if 'nif' in key.lower():
        diccionario_final["dni_cliente"] = dict_lineas[key]  
        break 



['0,XX €']
IMPORTE FACTURA: 90,70 €
['06 de marzo de 2014', '05 de abril de 2014']


In [29]:
# Mediante un enumerate extraemos todos los datos que sabemos que estan cerca unos de otros en la lista.
lista_consumo = []
lista_direccion = []
for indice, linea in enumerate(lista_lineas):
    if linea.lower() == 'consumo':
        lista_consumo.append(linea)
        lista_consumo.append(lista_lineas[indice+1]) 
        lista_consumo.append(lista_lineas[indice+2])

        for elemento in lista_consumo:
            if 'kwh' in elemento.lower():
                try:
                    diccionario_final["consumo_periodo"] = int(elemento.split()[0])
                except Exception:
                    continue    
    if 'dirección de suministro' in linea.lower():
        try:
            lista_direccion.append(linea) 
            lista_direccion.append(lista_lineas[indice+1]) 
            direccion_suministro = ' '.join(lista_direccion)
            direccion_dividir = direccion_suministro.split(':', 1)
            direccion = direccion_dividir[1].strip().split(',')
            diccionario_final["calle_cliente"] = direccion[0].strip() 
            diccionario_final["población_cliente"] = direccion[1].strip()  
        except Exception:
            direccion_entera = linea.split(':', 1)[1]
            direccion = re.split(r"(\d{5})", direccion_entera)
            diccionario_final["calle_cliente"] = direccion[0].strip() 
            diccionario_final["población_cliente"] = direccion[2].strip()  
            
# Se duplican los bucles for para poder romperlos después de la primera aparición en aquella información repetida a lo largo del pdf y que puede dar problemas.

for indice, linea in enumerate(lista_lineas):
    if diccionario_final["población_cliente"] in linea and len(linea) > len(diccionario_final["población_cliente"]):
        cp = linea.split()[0]
        try:
            cp_cliente = str(int(cp.strip()))
        except Exception:
            continue    
        diccionario_final["cp_cliente"] = cp_cliente
        break

nomi = pgeocode.Nominatim('es')
diccionario_final["provincia_cliente"] = nomi.query_postal_code(cp_cliente)['county_name']

for indice, linea in enumerate(lista_lineas):
    if diccionario_final["cif_comercializadora"] in linea:
        diccionario_final["nombre_comercializadora"] = lista_lineas[indice-1]
        direccion_comercializadora = lista_lineas[indice+1].split('-')
        
        diccionario_final["dirección_comercializadora"] = direccion_comercializadora[0].strip()[0:-5].strip()
        diccionario_final["población_comercializadora"] = direccion_comercializadora[-1].strip()
        diccionario_final["cp_comercializadora"] = direccion_comercializadora[0].strip()[-5:]
        break     
               

[' Calle los Jabalíes ', '43595', ' Prat de Comte']


In [30]:
diccionario_final 

{'nombre_cliente': 'NIEVES FRÍAS SARDINAS',
 'dni_cliente': '68623415V',
 'calle_cliente': 'Calle los Jabalíes',
 'cp_cliente': '43595',
 'población_cliente': 'Prat de Comte',
 'provincia_cliente': 'Tarragona',
 'nombre_comercializadora': '90,70 €',
 'cif_comercializadora': '',
 'dirección_comercializadora': 'GDF SUEZ TR',
 'cp_comercializadora': 'ADING',
 'población_comercializadora': 'GDF SUEZ TRADING',
 'provincia_comercializadora': '1, PLACE SAMUEL DE CHAMPLAIN (COURBEVOIE), 92400',
 'número_factura': '799889488',
 'inicio_periodo': '06.03.2014',
 'fin_periodo': '05.04.2014',
 'importe_factura': '90,70',
 'fecha_cargo': '10.04.2014',
 'consumo_periodo': '',
 'potencia_contratada': '2,43'}

In [ ]:
import json
with open("factura_555_meu.json", "w") as fp:
    json.dump(diccionario_final, fp, ensure_ascii=False)